Коротко: вместо «среднего за 30 дней с весами» используйте адаптивную модель заказа: (1) скоринговый фильтр всплесков, (2) экспоненциальное сглаживание с малым «периодом полураспада», (3) детектор смены режима (последние k дней против предыдущих), (4) расчёт уровня пополнения с учётом срока поставки и страхового запаса. Ниже — готовый рецепт и формулы.

# 1) Фильтр всплесков (winsorize)

Чтобы середина месяца с высоким спросом не «тянула» прогноз вверх:

* Оцените недавний верхний квантиль на окне 30 дней (например, 90-й перцентиль) и обрежьте дневные продажи сверху:

  $d_t^{*} = \min(d_t,\ Q_{0.90}^{(30)})$

Это сохраняет тренд, но гасит одиночные пики/акции.

# 2) Экспоненциальное сглаживание с полураспадом

Используйте EWMA, где более свежие дни получают экспоненциально большие веса.

* Полураспад $h$ (в днях): за $h$ дней вклад уменьшается вдвое. Рекомендую $h=5$–7 для чувствительности к последней неделе.

* Параметр сглаживания:

  $\alpha = 1 - 2^{-1/h}$

* Обновление уровня (оценка среднесуточного спроса):

  $\lambda_t = \alpha\, d_t^{*} + (1-\alpha)\,\lambda_{t-1}$

* Оценка дисперсии остатков для страхового запаса (тем же экспоненциальным весом, $\gamma \approx \alpha$):

  $e_t = d_t^{*} - \lambda_{t-1}$
  $s_t^2 = \gamma\, e_t^2 + (1-\gamma)\, s_{t-1}^2$

# 3) Детектор смены режима (быстрое «похолодание» спроса)

Сравните среднее за последние $k$ дней с предыдущим фоном:

* $\mu_{\text{recent}} = \text{avg}(d_{t-k+1}^{*},\ldots,d_t^{*})$
* $\mu_{\text{prev}} = \text{avg}(d_{t-k-m+1}^{*},\ldots,d_{t-k}^{*})$

Параметры: $k=5$, $m=15$.

Если наблюдается резкое падение,

$$
\frac{\mu_{\text{recent}}}{\mu_{\text{prev}}} < r_{\downarrow}\quad\text{(например }r_{\downarrow}=0.7\text{)}
$$

то «перезапускайте» состояние под текущий уровень:

* установить $\lambda_t \leftarrow \mu_{\text{recent}}$,
* временно сделать модель более «нервной» (уменьшите $h$ до 3–4 дней на 3–5 следующих обновлений).

Это предотвращает ситуацию, когда «жирная» середина месяца перевешивает последние 5–10 дней.

> Альтернатива ещё более робастная к пикам: **экспоненциально-взвешенная медиана** вместо среднего. Практически — посчитать медиану с весами $w_i \propto 2^{-(\text{возраст}_i/h)}$. Она меньше реагирует на кратковременные всплески, чем EWMA.

# 4) Уровень пополнения и количество заказа

Заказываем по политике «под уровень» (order-up-to):

* Пусть $L$ — срок поставки (в днях), $R$ — период между пересмотрами заказа (в днях).

* Прогноз спроса на горизонт $H=L+R$:

  $\widehat{D}_H = \lambda_t \cdot H$
  (если есть сильная недельная сезонность — умножайте $\lambda_t$ на индекс дня недели; см. примечание ниже)

* Страховой запас при целевом уровне сервиса $SL$ (например, 95% → $z\approx1{,}65$):

  $SS = z \cdot s_t \cdot \sqrt{H}$

* Целевой уровень запасов:

  $S = \widehat{D}_H + SS$

* Количество к заказу с учётом имеющихся остатков и уже в пути до даты $t+L$:

  $q = \max\big(0,\ S - \text{OnHand}_t - \text{OnOrder\_arriving\_by}(t+L)\big)$

### Значения по умолчанию

* Полураспад $h=5$ (последняя неделя решает), $\alpha = 1 - 2^{-1/5} \approx 0{,}129$.
  При «перезапуске» после падения спроса: временно $h=3$ ($\alpha\approx 0{,}206$).
* $k=5,\ m=15,\ r_{\downarrow}=0.7$.
* Уровень сервиса: 90% → $z=1.28$, 95% → $z=1.65$, 97.5% → $z=1.96$, 99% → $z=2.33$.

# 5) Недельная сезонность (если заметна)

Учтите день-недели: храните 7 сезонных индексов $s_{dow}$ (экспоненциально обновляемых) с нормировкой $\sum s_{dow}/7=1$. Тогда прогноз на день $t+i$:

$$
\widehat{d}_{t+i} = \lambda_t \cdot s_{\text{dow}(t+i)}
$$

и $\widehat{D}_H = \sum_{i=1}^H \widehat{d}_{t+i}$. Это часто снижает переоценку после «шумной» середины месяца.

---

## Псевдокод (Python-стиль)

```python
def order_qty(
    last_30_days, on_hand, on_order_arrivals_by_L, L, R,
    service_level=0.95, h=5, k=5, m=15, r_down=0.7,
    h_after_break=3, quantile_cap=0.90, use_weighted_median=False
):
    import math
    # 1) winsorize по верхнему квантилю
    q_cap = np.quantile(last_30_days, quantile_cap)
    d = np.minimum(last_30_days, q_cap)

    # 2) оценка уровня λ и дисперсии s^2
    def alpha_from_half_life(h): return 1 - 2**(-1.0/h)
    alpha = alpha_from_half_life(h)
    lam = np.mean(d[:5])  # инициализация
    s2 = np.var(d[:5]) if len(d) >= 5 else 0.0
    gamma = alpha
    for x in d:
        e = x - lam
        s2 = gamma*(e*e) + (1-gamma)*s2
        lam = alpha*x + (1-alpha)*lam

    # 3) детектор смены режима
    if len(d) >= k + m:
        mu_recent = np.mean(d[-k:])
        mu_prev   = np.mean(d[-(k+m):-k])
        if mu_prev > 0 and (mu_recent/mu_prev) < r_down:
            lam = mu_recent
            # сделать модель «нервнее» на ближайший горизонт
            alpha = alpha_from_half_life(h_after_break)
            # можно дополнительно снизить s2 (новый режим тише)
            s2 = max(s2 * 0.5, 1e-6)

    # (опционально) взвешенная медиана вместо EWMA
    if use_weighted_median:
        ages = np.arange(len(d))[::-1]  # 0 для последнего дня
        w = 0.5**(ages/float(h))        # экспоненциальные веса
        lam = weighted_quantile(d, 0.5, sample_weight=w)  # медиана

    # 4) заказ под уровень
    H = L + R
    z = {0.90:1.28, 0.95:1.65, 0.975:1.96, 0.99:2.33}.get(service_level, 1.65)
    demand_forecast = lam * H
    safety_stock = z * math.sqrt(max(s2, 1e-6)) * math.sqrt(H)
    S = demand_forecast + safety_stock

    q = max(0.0, S - on_hand - on_order_arrivals_by_L)
    return round(q)
```

> Реализация `weighted_quantile` — стандартная: сортировка по значению, накопление весов до 50% суммарного веса.

---

## Почему это решит вашу ситуацию

* **Winsorize + EWMA** гасит локальные пики в середине месяца.
* **Детектор смены режима** обрывает влияние «жирной» середины, если последние 5–10 дней явно ниже.
* **Страховой запас** не даёт «урезать до нуля» при шуме.
* **Период полураспада** настраивается предметно: чем быстрее меняется спрос, тем меньше $h$.

Если потребуется, могу адаптировать под ваш стек (ClickHouse-запросы + расчёты в Python/GS, индексы по дням недели, исключение промо-дней, отдельная логика для редких продаж/Croston).


____
____

Ок, делаем рабочий, самодостаточный вариант с понятными параметрами, комментариями и примером. Ошибка `weighted_quantile is not defined` была потому, что в псевдокоде она не была реализована. Ниже — **готовая функция** с документацией и реализацией `weighted_quantile`. Никаких внешних зависимостей (только стандартная библиотека).

---

### Что подставлять на вход

* `last_30_days`: список из 30 чисел — продажи по дням (шт/день) за последние 30 календарных дней (0 разрешён).
* `on_hand`: текущий остаток на складе (шт).
* `on_order_arrivals_by_L`: суммарное количество уже заказанного товара, которое успеет приехать **в течение срока поставки L** (шт).
* `L`: срок поставки в днях (lead time).
* `R`: периодичность пересмотра заказа в днях (обычно `1`, если проверяете ежедневно).
* Остальные параметры — тюнинг (дефолты ниже подходят большинству кейсов).

Рекомендуемые значения по умолчанию:
`service_level=0.95`, `h=5`, `k=5`, `m=15`, `r_down=0.7`, `h_after_break=3`, `quantile_cap=0.90`, `use_weighted_median=False`.

---

### Готовый код (скопируйте как есть)

```python
import math
from typing import List, Tuple, Dict, Any

def _percentile(a: List[float], p: float) -> float:
    """Непараметрический персентиль (линейная интерполяция). p в [0,1]."""
    if not a:
        return 0.0
    a_sorted = sorted(a)
    k = (len(a_sorted) - 1) * p
    f = math.floor(k)
    c = math.ceil(k)
    if f == c:
        return float(a_sorted[int(k)])
    return a_sorted[f] * (c - k) + a_sorted[c] * (k - f)

def _weighted_quantile(values: List[float], q: float, weights: List[float]) -> float:
    """
    Взвешенный квантиль q в [0,1]. Если веса нулевые/пусто, падение в невзвешенный персентиль.
    """
    if not values:
        return 0.0
    if not weights or sum(weights) <= 0:
        return _percentile(values, q)
    # Сортируем по значениям
    pairs = sorted(zip(values, weights), key=lambda x: x[0])
    vals = [v for v, _ in pairs]
    wts = [max(0.0, w) for _, w in pairs]
    total = sum(wts)
    target = q * total
    cum = 0.0
    for i, (v, w) in enumerate(zip(vals, wts)):
        prev = cum
        cum += w
        if cum >= target:
            # Линейная интерполяция внутри отрезка (если нужно)
            if w == 0:
                return float(v)
            frac = (target - prev) / w
            if i == 0:
                return float(v)
            return vals[i-1] * (1 - frac) + v * frac
    return float(vals[-1])

def _alpha_from_half_life(h: float) -> float:
    """Перевод половинного периода (в днях) в коэффициент EWMA."""
    h = max(1e-6, float(h))
    return 1.0 - 2.0 ** (-1.0 / h)

def order_qty(
    last_30_days: List[float],
    on_hand: float,
    on_order_arrivals_by_L: float,
    L: int,
    R: int,
    service_level: float = 0.95,
    h: float = 5.0,
    k: int = 5,
    m: int = 15,
    r_down: float = 0.7,
    h_after_break: float = 3.0,
    quantile_cap: float = 0.90,
    use_weighted_median: bool = False,
    return_details: bool = True,
) -> Tuple[int, Dict[str, Any]]:
    """
    Рассчитывает количество к заказу по политике "под уровень" с учётом нелинейного спроса.
    Шаги: winsorize, EWMA, детектор снижения режима, страховой запас.

    Параметры:
      last_30_days: продажи/день за последние 30 дней (список длиной <= 30, допускаются нули).
      on_hand: текущий складской остаток (шт).
      on_order_arrivals_by_L: уже заказано и приедет в течение L дней (шт).
      L: срок поставки (дни).
      R: период пересмотра (дни), обычно 1.
      service_level: целевой уровень сервиса (напр. 0.95 → z≈1.65).
      h: половинный период EWMA (чувствительность к последней неделе). Типично 5–7.
      k, m: окна детектора смены режима (последние k против предыдущих m).
      r_down: порог падения (если recent/prev < r_down → обнуляем состояние на низкий режим).
      h_after_break: временно более короткий half-life после детекта падения.
      quantile_cap: верхний квантиль для обрезки всплесков (winsorize), напр. 0.90.
      use_weighted_median: если True — использовать экспоненциально-взвешенную медиану вместо EWMA.
      return_details: если True, вернуть словарь с промежуточными расчётами.

    Возвращает:
      (q, details), где q — округлённое количество к заказу (int),
      details — словарь с диагностикой (уровень λ, дисперсия, прогноз, SS, S и т.п.).
    """
    # Санитация и обрезка входа
    d_raw = [max(0.0, float(x)) for x in last_30_days][-30:]
    if not d_raw:
        d_raw = [0.0]

    # 1) Winsorize по верхнему квантилю
    cap = _percentile(d_raw, max(0.0, min(1.0, quantile_cap)))
    d = [min(x, cap) for x in d_raw]

    # 2) EWMA: уровень λ и дисперсия остатков s^2
    alpha = _alpha_from_half_life(h)
    gamma = alpha
    # Инициализация λ и s2: среднее/дисперсия по первым min(5, len(d)) точкам
    init_n = min(5, len(d))
    lam = sum(d[:init_n]) / init_n
    # Несмещённость нам не критична: берём простую дисперсию
    mean0 = lam
    s2 = sum((x - mean0) ** 2 for x in d[:init_n]) / max(1, init_n)

    for x in d:
        e = x - lam
        s2 = gamma * (e * e) + (1.0 - gamma) * s2
        lam = alpha * x + (1.0 - alpha) * lam

    # 3) Детектор смены режима (падение спроса)
    mu_recent = mu_prev = ratio = None
    broke_regime = False
    if len(d) >= k + m and k > 0 and m > 0:
        mu_recent = sum(d[-k:]) / k
        mu_prev   = sum(d[-(k + m):-k]) / m
        if mu_prev > 0:
            ratio = mu_recent / mu_prev
            if ratio < r_down:
                # Перезапуск уровня на недавний режим
                lam = mu_recent
                alpha = _alpha_from_half_life(h_after_break)
                s2 = max(s2 * 0.5, 1e-6)
                broke_regime = True

    # (Опция) экспоненциально-взвешенная медиана вместо EWMA
    if use_weighted_median:
        ages = list(range(len(d)))[::-1]  # 0 для последнего дня
        weights = [0.5 ** (age / max(1e-6, h)) for age in ages]
        lam = _weighted_quantile(d, 0.5, weights)

    # 4) Заказ "под уровень"
    H = int(L) + int(R)
    H = max(1, H)
    z_table = {0.90: 1.28, 0.95: 1.65, 0.975: 1.96, 0.99: 2.33}
    # Берём ближайшее известное z или дефолт 1.65
    z = min(z_table.items(), key=lambda kv: abs(kv[0] - service_level))[1]

    demand_forecast = lam * H
    safety_stock = z * math.sqrt(max(s2, 1e-12)) * math.sqrt(H)
    S = demand_forecast + safety_stock

    q = max(0.0, S - float(on_hand) - float(on_order_arrivals_by_L))
    q_rounded = int(round(q))

    details = {
        "lambda_level": lam,
        "variance_s2": s2,
        "alpha": alpha,
        "cap_quantile": quantile_cap,
        "cap_value": cap,
        "detector": {
            "broke_regime": broke_regime,
            "k": k, "m": m, "r_down": r_down,
            "mu_recent": mu_recent, "mu_prev": mu_prev, "ratio": ratio,
        },
        "h": h, "h_after_break": h_after_break,
        "H": H, "z": z,
        "demand_forecast_H": demand_forecast,
        "safety_stock": safety_stock,
        "target_S": S,
        "on_hand": float(on_hand),
        "on_order_arrivals_by_L": float(on_order_arrivals_by_L),
        "q_continuous": q,
    }

    return (q_rounded, details) if return_details else (q_rounded, {})
```

---

### Мини-пример использования

```python
# Продажи за 30 дней: низко → пик в середине → снова низко
last_30 = [2,2,3,1,0,  2,3,2,3,2,  10,12,9,11,10,  3,2,2,1,2,  1,1,0,2,1,  1,2,1,1]
on_hand = 120
on_order_arrivals_by_L = 50
L = 7   # доставка 7 дней
R = 1   # проверяем каждый день

q, info = order_qty(
    last_30, on_hand, on_order_arrivals_by_L, L, R,
    service_level=0.95,
    h=5, k=5, m=15, r_down=0.7, h_after_break=3,
    quantile_cap=0.90,
    use_weighted_median=False,   # включите True, если хотите медиану
    return_details=True
)

print("Количество к заказу:", q)
print("Ключевые детали:", {k: info[k] for k in ["lambda_level","demand_forecast_H","safety_stock","target_S"]})
print("Детектор смены режима:", info["detector"])
```

---

### Как выбрать значения параметров (кратко)

* `L`: ваш реальный срок поставки в днях. Если плавает — берите верхнюю границу SLA.
* `R`: как часто вы пересчитываете заказ (дней). Ежедневно → `R=1`.
* `service_level`: 0.90/0.95/0.975/0.99 — под ваш SLA отгрузки.
* `h` (half-life): чем быстрее меняется спрос, тем меньше `h`. Типично 5–7.
* `k, m, r_down`: детектор падения. База — `k=5`, `m=15`, `r_down=0.7`.
* `quantile_cap`: обрезка всплесков (акции). 0.9–0.95.
* `use_weighted_median`: ставьте `True`, если много кратких промо-пиков — медиана ещё устойчивее к разовым всплескам.

---

### Почему ошибки больше не будет

Раньше функция ссылалась на `weighted_quantile`, но её не было. В коде выше реализована `_weighted_quantile`, так что:

* при `use_weighted_median=False` она вообще не вызывается;
* при `use_weighted_median=True` — вызов есть и работает.

---

### Частые вопросы

**Как посчитать `on_order_arrivals_by_L`?**
Суммируйте все позиции в пути с датой прибытия ≤ (сегодня + `L` дней). Если недоступны точные ETA — используйте ожидаемую дату как дата заказа + плановый транзит.

**Что если продаж < 30 дней?**
Передайте сколько есть (хоть 7–14). Функция сама обрежет до последних и корректно инициализируется.

**Можно ли учесть сезонность по дням недели?**
Да, следующий шаг — хранить 7 индексов и корректировать `demand_forecast_H`. Если нужно — дам готовый блок к этому коду.

---

Если хотите, могу показать, как это встроить в ваш текущий пайплайн (ClickHouse → Python → Google Sheets) на одном SKU или батчем по всем SKU.


____
____

Коротко: «под уровень» (= **order-up-to**) — вы каждый раз добираете запас не «на N дней», а до целевого уровня $S$, рассчитанного под горизонт $H=L+R$, где
$L$ — срок поставки (lead time), $R$ — периодичность пересмотра (как часто вы делаете заказ).
То есть заказ должен **покрыть спрос на ближайшие $L+R$ дней** с нужной вероятностью сервиса.

### Термины

* $L$: дней до прихода текущего заказа.
* $R$: через сколько дней вы снова будете пересматривать заказ (ежедневно → $R=1$, раз в неделю → $R=7$).
* $H = L + R$: горизонт, под который рассчитывается уровень.
* **Позиция запаса** (inventory position) на момент заказа:
  $\text{IP} = \text{OnHand} + \text{OnOrder} - \text{Backorders}$.

### Что значит «расчёт под уровень»

1. Считаем целевой уровень $S$ «на горизонте»:

$$
S = \underbrace{\mathbb{E}[D_{H}]}_{\text{ожидаемый спрос за }H} \;+\; 
\underbrace{z\cdot \sigma \sqrt{H}}_{\text{страховой запас под уровень сервиса}}
$$

где $\mathbb{E}[D_H] \approx \lambda\cdot H$ (ваш прогноз среднесуточного спроса),
$\sigma$ — оценка суточного разброса, $z$ — квантиль сервиса (95% → $z\approx1{,}65$).

2. Добираем «до уровня»:

$$
q = \max\bigl(0,\; S - \text{IP}\bigr).
$$

Это и есть «под уровень»: сколько не хватает, чтобы довести позицию запаса до целевого $S$.

### На какой промежуток делается поставка

Не на фиксированные «N дней», а на **$H=L+R$ дней**.
Интерпретация: заказ сегодня должен обеспечить, чтобы к моменту прихода через $L$ дней и до следующего пересмотра ещё $R$ дней вероятность дефицита была не выше целевой (через страховой запас).

### Практический нюанс про «что вычитать»

Классическая формула берёт **весь** $\text{OnOrder}$ (все заказы в пути).
На практике безопаснее вычитать только то, что **успеет прийти до $t+L$** (как в моём коде параметр `on_order_arrivals_by_L`), если у вас есть дальние поставки с ETA сильно позже $L$. Это избегает недозаказа «на ближний горизонт».

### Мини-пример

Пусть $\lambda=10$ шт/день, $\sigma=6$ шт/день, $L=7$, $R=1$ → $H=8$.
Тогда:

* Страховой запас: $SS = 1{,}65 \cdot 6 \cdot \sqrt{8} \approx 28$.
* Ожидаемый спрос: $80$.
* Целевой уровень: $S \approx 108$.

Заказ:

* если $\text{OnHand}=60$ и ничего не приедет до $L$: $q = 108-60=48$;
* если $\text{OnHand}=60$ и до $L$ приедет ещё 20: $q = 108-60-20=28$.

Так вы всегда «добираете» до $S$, а не пытаетесь угадать «поставка на 10/14 дней».
В вашем коде это соответствует строкам, где $H=L+R$, $S=\lambda H + z\sqrt{s^2}\sqrt{H}$, а $q=\max(0, S - \text{OnHand} - \text{OnOrder\_arrivals\_by\_L})$.


____
____

### Суммарный спрос на горизонте H=L+R (факт vs прогноз)

<img src="data/img/output.png" width=1979 height=980 />

### Факт H-спроса vs уровень S (T1, T2)

<img src="data/img/output2.png" width=1979 height=980 />

Сделал 3 теста на вашем ряду (файл `/mnt/data/fjr_gpt.xlsx`, лист «Лист1»). Предположения для бэктеста: горизонт $H=L+R=7+1=8$ дней, расчет ежедневно, склад/заказы в пути **не учитывал** (оценивали, покрывает ли «уровень S» фактический спрос на горизонте при $IP=0$). Окно истории — последние 30 дней на каждый день расчёта. Получилось 52 точки сравнения (с 2025-01-31 по 2025-03-22).

Сводка по трём вариантам (таблица «Итоги 3 тестов (резюме)» открыта выше; выгрузки ниже):

* T0\_baseline\_mean (наивное среднее 30 дней):
  MAE 307.94, MAPE 1.81, Bias +302.10, Coverage 1.000, P75 overage ≈ 1063.7.
  Итог: грубый пере-прогноз (перезаказ), не подходит.

* T1\_EWMA (winsorize 0.90 + EWMA + детектор падения):
  MAE 57.27, MAPE 0.391, Bias +23.36, Coverage 0.885, P75 overage ≈ 234.7.
  Итог: точность заметно лучше, перезаказ умеренный; покрытие ниже целевых 95%.

* T2\_EWMed (то же, но экспоненциально-взвешенная медиана):
  MAE 53.63, MAPE 0.380, Bias +29.82, Coverage 0.904, P75 overage ≈ 208.6.
  Итог: ещё немного лучше по точности, покрытие ≈90%.

Графики выше:

1. факт H-спроса vs прогнозы (видно, что T0 сильно завышает из-за прошлых «жирных» периодов);
2. факт H-спроса vs уровень S (T1, T2), где видно величину «зазора» S–факт.

### Вывод по пригодности к вашему кейсу

* Код **решает основную проблему**: устраняет систематический пере-заказ из-за «жирной середины» месяца. И T1, и T2 значительно лучше наивного среднего.
* Если вам нужен фактический уровень сервиса порядка **90%**, T2 уже ок (Coverage ≈ 0.904).
* Если требуется **\~95%**, нужно увеличить «подушку»: либо поднять `service_level` до 0.975 (z≈1.96), либо уменьшить half-life после обрыва тренда настроечно и/или ослабить обрезку всплесков (например, `quantile_cap` 0.95). Это повысит Coverage, но немного увеличит перезаказ.

### Что именно я запускал

* Горизонт $H=8$ (L=7, R=1), `service_level=0.95`, `h=5`, `k=5`, `m=15`, `r_down=0.7`, `h_after_break=3`, `quantile_cap=0.90`.
* Тесты:
  T0 — среднее за 30 дней;
  T1 — наш вариант с EWMA;
  T2 — наш вариант с экспоненциально-взвешенной медианой.

### Файлы с результатами

* [Скачать детали бэктеста (по дням)](sandbox:/mnt/data/order_model_backtest_details.csv)
* [Скачать сводку по моделям](sandbox:/mnt/data/order_model_backtest_summary.csv)

Если дадите фактические параметры склада (OnHand, OnOrder с ETA, целевой SLA сервиса) — пересчитаю уже «по-взрослому» с симуляцией запасов и реальными IP, чтобы оценить не только покрытие S, но и ожидаемые out-of-stock и излишки.


___
___

Готово. Я прогнал симуляцию «под уровень» на вашем файле c продажами и остатками. Ниже — сухой итог и ссылки на выгрузки.

## Что именно я смоделировал

* Пересмотр ежедневно (R=1).
* Срок поставки: два варианта

  1. **const8**: фиксированный L=8 дней;
  2. **uniform\_6\_10**: случайный L∈\[6;10] (равновероятно).
     В формуле уровня учёл дисперсию срока поставки:
     $S=\lambda\cdot(\mathbb{E}[L]+R)+z\cdot\sqrt{s^2\cdot(\mathbb{E}[L]+R)+\lambda^2\cdot\mathrm{Var}(L)}$.
* Два прогноза суточного спроса:
  **T1\_EWMA** (winsorize 0.90 + EWMA + детектор падения) и **T2\_EWMed** (то же, но взвешенная медиана).
* Начальный «прогрев» важен. Показал результаты **(A)** на всём периоде «как есть» (с первых дней история ещё короткая ⇒ риск промаха) и **(B)** с «прогревом» — оценка/сравнение с **31-го дня**, когда история уже есть (корректнее для эксплуатации).

---

## Результаты

### (A) Весь период с 2025-01-01 (без прогрева)

* Факт (по файлу): fill rate 100%, средний остаток **1631** шт, поставок (по расчёту прибытия) **80** дней.
* Модель (T1/T2): средний остаток **1065–1315** шт, но fill rate **\~87–90%** (стоки в начале периода из-за короткой истории и отсутствия «старта» пайплайна).
* Детали: [CSV (full)](sandbox:/mnt/data/sim_vs_actual_metrics.csv), поминутные логи симуляций:

  * T1 const8: [CSV](sandbox:/mnt/data/sim_T1_EWMA_const8.csv)
  * T2 const8: [CSV](sandbox:/mnt/data/sim_T2_EWMed_const8.csv)
  * T1 uniform\_6\_10: [CSV](sandbox:/mnt/data/sim_T1_EWMA_uniform_6_10.csv)
  * T2 uniform\_6\_10: [CSV](sandbox:/mnt/data/sim_T2_EWMed_uniform_6_10.csv)
  * Факт (с расчётом приходов): [CSV](sandbox:/mnt/data/actual_with_inferred_inbound.csv)

> Причина просадки сервиса здесь тривиальна: с «нуля» нет 30-дневной истории и нет «заполненного» пайплайна — первые заказы приезжают через 6–10 дней, а остаток успевает просесть. В эксплуатации это решается «инициализирующим добором под уровень» или стартом после накопления истории.

---

### (B) Корректная оценка **с 2025-01-31** (после 30 дней истории, старт с фактического остатка)

Период сравнения: 2025-01-31…2025-03-31.

| Вариант             | Lead time      | Fill rate | Ср. остаток | Дней с заказом | Всего заказано |
| ------------------- | -------------- | --------: | ----------: | -------------: | -------------: |
| **Факт** (из файла) | наблюдаемый    |  **100%** |    **1391** |         **54** |           2604 |
| **T1\_EWMA\_W**     | const8         |  **100%** |     **212** |             29 |            853 |
| **T2\_EWMed\_W**    | const8         |  **100%** |     **229** |             26 |            851 |
| **T1\_EWMA\_W**     | uniform\_6\_10 |  **100%** |     **225** |             26 |            865 |
| **T2\_EWMed\_W**    | uniform\_6\_10 |  **100%** |     **258** |             23 |            863 |

Файлы:

* Итоги по окну с прогревом: [CSV](sandbox:/mnt/data/sim_warmup_metrics.csv)
  (пер-симуляция: [T1 const8](sandbox:/mnt/data/sim_warm_T1_EWMA_const8.csv), [T2 const8](sandbox:/mnt/data/sim_warm_T2_EWMed_const8.csv), [T1 uni(6–10)](sandbox:/mnt/data/sim_warm_T1_EWMA_uniform_6_10.csv), [T2 uni(6–10)](sandbox:/mnt/data/sim_warm_T2_EWMed_uniform_6_10.csv))
* Метрики факта на том же окне: [CSV](sandbox:/mnt/data/actual_window_metrics.csv)

**Вывод по (B):** метод работает корректно для вашего ряда. При ежедневном пересмотре и L∈\[6;10] он держит **100% fill rate** на окне оценки, при этом **средний остаток ниже фактического \~в 5–6 раз** и реже вызывает поставки (крупнее и реже вместо мелких и частых).

---

## Итог

* **Да, метод подходит** для вашего сценария — при стартовом «прогреве» или однократном доборе «до уровня» на старте.
* Если нужен сервис >99% при переменном L — поднимайте `service_level` до 0.99 и/или используйте более консервативный L\_guard=10 при расчёте $q$ (я в модели уже вычитаю поставки только в пределах L\_guard). Это увеличит S, но поднимет средний остаток.

## Что можно докрутить (по мере необходимости)

1. Индексы дня недели (сезонность) в прогнозе.
2. Исключение промо-дней по календарю.
3. Использование реальных ETA — тогда вычитание `on_order_arrivals_by_L` будет ровно соответствовать вашему пайплайну.
4. При вводе в эксплуатацию — выполнить разовый «bootstrap-заказ» $q_0=\max(0,S-IP)$ в первый день, чтобы исключить стартовые стоки.

Если хотите — прогоню ещё один тест с вашим целевым SLA (например, 97.5% или 99%) и/или с учётом реальных графиков поставок по ETA.


In [ ]:
import math
from typing import List, Tuple, Dict, Any

def _percentile(a: List[float], p: float) -> float:
    """Непараметрический персентиль (линейная интерполяция). p в [0,1]."""
    if not a:
        return 0.0
    a_sorted = sorted(a)
    k = (len(a_sorted) - 1) * p
    f = math.floor(k)
    c = math.ceil(k)
    if f == c:
        return float(a_sorted[int(k)])
    return a_sorted[f] * (c - k) + a_sorted[c] * (k - f)

def _weighted_quantile(values: List[float], q: float, weights: List[float]) -> float:
    """
    Взвешенный квантиль q в [0,1]. Если веса нулевые/пусто, падение в невзвешенный персентиль.
    """
    if not values:
        return 0.0
    if not weights or sum(weights) <= 0:
        return _percentile(values, q)
    # Сортируем по значениям
    pairs = sorted(zip(values, weights), key=lambda x: x[0])
    vals = [v for v, _ in pairs]
    wts = [max(0.0, w) for _, w in pairs]
    total = sum(wts)
    target = q * total
    cum = 0.0
    for i, (v, w) in enumerate(zip(vals, wts)):
        prev = cum
        cum += w
        if cum >= target:
            # Линейная интерполяция внутри отрезка (если нужно)
            if w == 0:
                return float(v)
            frac = (target - prev) / w
            if i == 0:
                return float(v)
            return vals[i-1] * (1 - frac) + v * frac
    return float(vals[-1])

def _alpha_from_half_life(h: float) -> float:
    """Перевод половинного периода (в днях) в коэффициент EWMA."""
    h = max(1e-6, float(h))
    return 1.0 - 2.0 ** (-1.0 / h)

def order_qty(
    last_30_days: List[float],
    on_hand: float,
    on_order_arrivals_by_L: float,
    L: int,
    R: int,
    service_level: float = 0.95,
    h: float = 5.0,
    k: int = 5,
    m: int = 15,
    r_down: float = 0.7,
    h_after_break: float = 3.0,
    quantile_cap: float = 0.90,
    use_weighted_median: bool = False,
    return_details: bool = True,
) -> Tuple[int, Dict[str, Any]]:
    """
    Рассчитывает количество к заказу по политике "под уровень" с учётом нелинейного спроса.
    Шаги: winsorize, EWMA, детектор снижения режима, страховой запас.

    Параметры:
      last_30_days: продажи/день за последние 30 дней (список длиной <= 30, допускаются нули).
      on_hand: текущий складской остаток (шт).
      on_order_arrivals_by_L: уже заказано и приедет в течение L дней (шт).
      L: срок поставки (дни).
      R: период пересмотра (дни), обычно 1.
      service_level: целевой уровень сервиса (напр. 0.95 → z≈1.65).
      h: половинный период EWMA (чувствительность к последней неделе). Типично 5–7.
      k, m: окна детектора смены режима (последние k против предыдущих m).
      r_down: порог падения (если recent/prev < r_down → обнуляем состояние на низкий режим).
      h_after_break: временно более короткий half-life после детекта падения.
      quantile_cap: верхний квантиль для обрезки всплесков (winsorize), напр. 0.90.
      use_weighted_median: если True — использовать экспоненциально-взвешенную медиану вместо EWMA.
      return_details: если True, вернуть словарь с промежуточными расчётами.

    Возвращает:
      (q, details), где q — округлённое количество к заказу (int),
      details — словарь с диагностикой (уровень λ, дисперсия, прогноз, SS, S и т.п.).
    """
    # Санитация и обрезка входа
    d_raw = [max(0.0, float(x)) for x in last_30_days][-30:]
    if not d_raw:
        d_raw = [0.0]

    # 1) Winsorize по верхнему квантилю
    cap = _percentile(d_raw, max(0.0, min(1.0, quantile_cap)))
    d = [min(x, cap) for x in d_raw]

    # 2) EWMA: уровень λ и дисперсия остатков s^2
    alpha = _alpha_from_half_life(h)
    gamma = alpha
    # Инициализация λ и s2: среднее/дисперсия по первым min(5, len(d)) точкам
    init_n = min(5, len(d))
    lam = sum(d[:init_n]) / init_n
    # Несмещённость нам не критична: берём простую дисперсию
    mean0 = lam
    s2 = sum((x - mean0) ** 2 for x in d[:init_n]) / max(1, init_n)

    for x in d:
        e = x - lam
        s2 = gamma * (e * e) + (1.0 - gamma) * s2
        lam = alpha * x + (1.0 - alpha) * lam

    # 3) Детектор смены режима (падение спроса)
    mu_recent = mu_prev = ratio = None
    broke_regime = False
    if len(d) >= k + m and k > 0 and m > 0:
        mu_recent = sum(d[-k:]) / k
        mu_prev   = sum(d[-(k + m):-k]) / m
        if mu_prev > 0:
            ratio = mu_recent / mu_prev
            if ratio < r_down:
                # Перезапуск уровня на недавний режим
                lam = mu_recent
                alpha = _alpha_from_half_life(h_after_break)
                s2 = max(s2 * 0.5, 1e-6)
                broke_regime = True

    # (Опция) экспоненциально-взвешенная медиана вместо EWMA
    if use_weighted_median:
        ages = list(range(len(d)))[::-1]  # 0 для последнего дня
        weights = [0.5 ** (age / max(1e-6, h)) for age in ages]
        lam = _weighted_quantile(d, 0.5, weights)

    # 4) Заказ "под уровень"
    H = int(L) + int(R)
    H = max(1, H)
    z_table = {0.90: 1.28, 0.95: 1.65, 0.975: 1.96, 0.99: 2.33}
    # Берём ближайшее известное z или дефолт 1.65
    z = min(z_table.items(), key=lambda kv: abs(kv[0] - service_level))[1]

    demand_forecast = lam * H
    safety_stock = z * math.sqrt(max(s2, 1e-12)) * math.sqrt(H)
    S = demand_forecast + safety_stock

    q = max(0.0, S - float(on_hand) - float(on_order_arrivals_by_L))
    q_rounded = int(round(q))

    details = {
        "lambda_level": lam,
        "variance_s2": s2,
        "alpha": alpha,
        "cap_quantile": quantile_cap,
        "cap_value": cap,
        "detector": {
            "broke_regime": broke_regime,
            "k": k, "m": m, "r_down": r_down,
            "mu_recent": mu_recent, "mu_prev": mu_prev, "ratio": ratio,
        },
        "h": h, "h_after_break": h_after_break,
        "H": H, "z": z,
        "demand_forecast_H": demand_forecast,
        "safety_stock": safety_stock,
        "target_S": S,
        "on_hand": float(on_hand),
        "on_order_arrivals_by_L": float(on_order_arrivals_by_L),
        "q_continuous": q,
    }

    return (q_rounded, details) if return_details else (q_rounded, {})


In [ ]:

last_30 = [
    26, 25, 28, 20, 22, 9, 19, 15, 9, 11, 26, 37, 32, 37, 35, 24, 23, 21, 40, 21, 11, 14, 17, 22, 27, 25, 25, 29, 24, 17
]
last_30.reverse()
on_hand = 1860

In [ ]:
last_30 = [
    26, 33, 36, 56, 66, 51, 51, 121, 217, 296, 263, 313, 285, 188, 237, 265, 202, 139, 202, 156, 185, 185, 123, 132, 87, 108, 50, 64, 70, 68
]
last_30.reverse()
# on_hand = 1484
on_hand = 17

In [ ]:
# Продажи за 30 дней: низко → пик в середине → снова низко
# last_30 = []
# on_hand = 1860
on_order_arrivals_by_L = 0
L = 7   # доставка 7 дней
R = 1   # проверяем каждый день

q, info = order_qty(
    last_30, on_hand, on_order_arrivals_by_L, L, R,
    service_level=0.95,
    h=5, k=5, m=15, r_down=0.7, h_after_break=3,
    quantile_cap=0.90,
    use_weighted_median=False,   # включите True, если хотите медиану
    return_details=True
)

print("Количество к заказу:", q)
print("Ключевые детали:", {k: info[k] for k in ["lambda_level","demand_forecast_H","safety_stock","target_S"]})
print("Детектор смены режима:", info["detector"])